# Proyecto final
## Nombre: Go_Fin
    - Go_Fin es una herramienta basada en las empresas que componen el índice S&P500 y permite realizar un análisis de estados financieros de alguna entidad con base al sector e industria

Las paqueterías a usar durante todo el proyecto son las siguientes:
(la falta de alguna puede afectar el resultado)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from functools import reduce

Origen de los datos:  Capital IQ (plug-in excel) y fórmulas de excel

Existen 4 datasets que consisten en lo siguiente:
1. balance: estado de situación financiera o balance general
2. income: estado de resultados
3. capital_structure: estructura de deuda de la entidad


In [2]:
balance = pd.read_csv('Datasets/bs.csv', index_col = 0)
income = pd.read_csv('Datasets/is.csv', index_col = 0)
capital_structure = pd.read_csv('Datasets/cs.csv', index_col = 0)
company_info = pd.read_csv('Datasets/company_information.csv')

Todos los datasets tuvieron un previa manipulación en RStudio y debido al cambio de entorno se decidió no realizar un reproceso de lo ya hecho

Previsualización de los tipos de datos de cada DataSet:

In [3]:
balance.dtypes

Ticker                       object
Symbol                       object
Stock                        object
Sector                       object
Year_as                      object
Year                          int64
Company_Name                 object
CASH                        float64
ST_INVESTMENTS              float64
RECEIVABLES                 float64
INVENTORY                   float64
OTHER_CURRENT_ASSETS        float64
TOTAL_CURRENT_ASSETS        float64
PP&E                        float64
DEPRECIATION                float64
LT_INVESTMENTS              float64
INTANGIBLES_GW              float64
OTHER_NON_CURRENT_ASSETS    float64
NON_CURRENT_ASSETS          float64
TOTAL_ASSETS                float64
PAYABLES                    float64
ST_DEBT                     float64
CPLTD_DEBT                  float64
CPLTD_LEASES                float64
CURRENT_LIAB                float64
LT_DEBT                     float64
LT_LEASES                   float64
OTHER_NON_CURRENT_LIAB      

In [4]:
income.dtypes

Ticker                  object
Symbol                  object
Stock                   object
Sector                  object
Year                     int64
Year_as                 object
Company_Name            object
TOTAL_REVENUES         float64
COGS                   float64
GROSS_PROFIT           float64
SG_A                   float64
R_D                    float64
D_A                    float64
A_INTANGIBLES          float64
NET_INTEREST_EXP       float64
UNUSUAL_ITEMS          float64
EBT                    float64
TAX_EXPENSE            float64
E_CONT_OPER            float64
E_DISC_OPER            float64
EXTR_ITEMS               int64
OTHER_ADJUSTMENTS      float64
M_A_COST               float64
NET_INCOME_INCL_EI     float64
NET_INCOME_EXCL_EI     float64
RENTAL_EXPENSE         float64
INTEREST_LT            float64
TAXES                  float64
DEFERRED_TAX           float64
EBIT                   float64
EBITDA                 float64
CONSOLIDATED_EBITDA    float64
EBITDAR 

In [5]:
company_info.dtypes

Identifier           object
Stock                object
Symbol               object
Company_Name         object
GICS_Sector          object
GICS_Sub-Industry    object
Region               object
States               object
Date_first_added     object
CIK                   int64
Founded               int64
IPO Year             object
Country              object
Volume               object
Market Cap           object
dtype: object

In [6]:
company_info.rename(columns = {"GICS_Sub-Industry": "GICS_SubIndustry"}, inplace=True)

In [7]:
ticker_1 = company_info.Symbol.unique()
ticker_symbol = ticker_1.tolist()
company_Name = company_info.Company_Name.unique()
sector = company_info.GICS_Sector.unique()
subsector = company_info.GICS_SubIndustry.unique()

Este código no se puede borrar ya que es el que valida información en el programa:

In [8]:
list_year = [2015,2016,2017,2018,2019,2020]

Conociendo las columnas de cada DataSet:

In [9]:
balance.columns

Index(['Ticker', 'Symbol', 'Stock', 'Sector', 'Year_as', 'Year',
       'Company_Name', 'CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY',
       'OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION',
       'LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS',
       'NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT',
       'CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES',
       'OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB',
       'RETAINED_EAR', 'TOTAL_EQUITY'],
      dtype='object')

In [10]:
income.columns

Index(['Ticker', 'Symbol', 'Stock', 'Sector', 'Year', 'Year_as',
       'Company_Name', 'TOTAL_REVENUES', 'COGS', 'GROSS_PROFIT', 'SG_A', 'R_D',
       'D_A', 'A_INTANGIBLES', 'NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT',
       'TAX_EXPENSE', 'E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS',
       'OTHER_ADJUSTMENTS', 'M_A_COST', 'NET_INCOME_INCL_EI',
       'NET_INCOME_EXCL_EI', 'RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES',
       'DEFERRED_TAX', 'EBIT', 'EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR'],
      dtype='object')

**Para poder realizar los dataset de ratios financieros se ocupó como fuente la siguiente:**
Fuente = Corporate Finance Institute

1. Ratios de Leverage (apalancamiento)
Debt to assets = Debt / Assets
Debt to equity = Debt / Equity
Debt to Capital Ratio = Debt / Capital
Debt to EBITDA = Debt / EBITDA
Asset to Equity Ratio = Assets / Equity
Total Liabilities / Net Worth

2. Ratios de coverage (cobertura)
Interest Coverage = Operating Income / Interest Expense
Debt Service Coverage Ratio = Operating Income / Total Debt Service
Cash Coverage Ratio = Total Cash / Interest Expense
Asset Coverage Ratio = (Total Assets - Intangible Assets) - (Current Liabilities -Short Term Debt) / Interest Expense

3. Ratios de Liquidity (liquidez)
Current ratio = current assets / current liabilities
Acid test = Current assets - Inventories / Current liabilities
Quick Ratio = Cash + accounts receivables  + markeatable securities / current liabilities
Cash ratio = Cash + Marketable securities / current liabilities

4. Profitability Ratios
Gross Profit Margin
EBITDA margin
Operating profit margin
Net Profit Margin
Return on assets
Return on Equity
ROIC

5. Market Value Ratios
Book value per share ratio = (Shareholders Equity - Preffered equity) / Total shares outstanding
Dividend yiel ratio = Dividen per share / Share price
Earnings per share = Net earnings / total shares outstanding
Price-earnings ratio = Share Price / Earnings per share

**Aquí inicia la elaboración de los dataset**

In [11]:
df_ratio = balance.merge(income, how = 'inner', left_on = ['Ticker', 'Symbol', 'Year'], right_on = ['Ticker', 'Symbol', 'Year'])
df_ratio = pd.merge(df_ratio, company_info[['Symbol','GICS_SubIndustry']], on = ['Symbol'] )
df_ratio = df_ratio.drop(['Stock_y', 'Sector_y', 'Year_as_y','Company_Name_y'], axis = 1)
df_ratio.columns

Index(['Ticker', 'Symbol', 'Stock_x', 'Sector_x', 'Year_as_x', 'Year',
       'Company_Name_x', 'CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY',
       'OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION',
       'LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS',
       'NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT',
       'CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES',
       'OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB',
       'RETAINED_EAR', 'TOTAL_EQUITY', 'TOTAL_REVENUES', 'COGS',
       'GROSS_PROFIT', 'SG_A', 'R_D', 'D_A', 'A_INTANGIBLES',
       'NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT', 'TAX_EXPENSE',
       'E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS', 'OTHER_ADJUSTMENTS',
       'M_A_COST', 'NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI',
       'RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES', 'DEFERRED_TAX', 'EBIT',
       'EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR', 'GICS_SubIndustry'],
      dtype='ob

In [12]:
df_ratio['TOTAL_DEBT'] = df_ratio['ST_DEBT'] + df_ratio['CPLTD_DEBT'] + df_ratio['CPLTD_LEASES'] + df_ratio['LT_DEBT'] + df_ratio['LT_LEASES']

In [13]:
df_ratio1 = df_ratio.copy(deep = True)

***LEVERAGE***

In [14]:
df_ratio1['DEBT_TO_ASSETS'] = round(df_ratio1['TOTAL_DEBT'] / df_ratio1['TOTAL_ASSETS'],2)
df_ratio1['DEBT_TO_EQUITY'] = round(df_ratio1['TOTAL_DEBT'] / df_ratio1['TOTAL_EQUITY'],2)
df_ratio1['DEBT_TO_CAPITAL'] = round(df_ratio1['TOTAL_DEBT'] / (df_ratio1['TOTAL_EQUITY']-df_ratio1['RETAINED_EAR']),2)
df_ratio1['DEBT_TO_EBITDA'] = round(df_ratio1['TOTAL_DEBT'] / df_ratio1['CONSOLIDATED_EBITDA'],2)
df_ratio1['DEBT_TO_EBITDAR'] = round(df_ratio1['TOTAL_DEBT'] / df_ratio1['EBITDAR'],2)
df_ratio1['ASSET_TO_EQUITY'] = round(df_ratio1['TOTAL_ASSETS'] / df_ratio1['TOTAL_EQUITY'],2)
df_ratio1['LIABILITIES_TO_EQUITY'] = round(df_ratio1['TOTAL_LIAB'] / df_ratio1['TOTAL_EQUITY'],2)

In [15]:
leverage = df_ratio1.drop(['CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY','OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION','LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS','NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT','CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES','OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB','RETAINED_EAR', 'TOTAL_EQUITY', 'TOTAL_REVENUES', 'COGS','GROSS_PROFIT', 'SG_A', 'R_D', 'D_A', 'A_INTANGIBLES','NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT', 'TAX_EXPENSE','E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS', 'OTHER_ADJUSTMENTS','M_A_COST', 'NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI','RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES', 'DEFERRED_TAX', 'EBIT','EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR'], axis = 1)
leverage.dtypes

Ticker                    object
Symbol                    object
Stock_x                   object
Sector_x                  object
Year_as_x                 object
Year                       int64
Company_Name_x            object
GICS_SubIndustry          object
TOTAL_DEBT               float64
DEBT_TO_ASSETS           float64
DEBT_TO_EQUITY           float64
DEBT_TO_CAPITAL          float64
DEBT_TO_EBITDA           float64
DEBT_TO_EBITDAR          float64
ASSET_TO_EQUITY          float64
LIABILITIES_TO_EQUITY    float64
dtype: object

***COVERAGE***

In [16]:
df_ratio2 = df_ratio.copy(deep = True)

In [17]:
df_ratio2['INTEREST_COVERAGE'] = round(df_ratio2['EBIT'] / df_ratio2['NET_INTEREST_EXP'],2)
df_ratio2['DEBT_SERVICE_COVERAGE'] = round(df_ratio2['EBIT'] / df_ratio2['TOTAL_DEBT'],2)
df_ratio2['CASH_COVERAGE'] = round(df_ratio2['CASH'] / df_ratio2['NET_INTEREST_EXP'],2)
df_ratio2['ASSET_COVERAGE'] = round((df_ratio2['TOTAL_ASSETS'] - df_ratio2['INTANGIBLES_GW']) - (df_ratio2['CURRENT_LIAB'] - df_ratio2['ST_DEBT'])   / df_ratio2['NET_INTEREST_EXP'],2)

In [18]:
coverage = df_ratio2.drop(['CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY','OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION','LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS','NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT','CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES','OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB','RETAINED_EAR', 'TOTAL_EQUITY', 'TOTAL_REVENUES', 'COGS','GROSS_PROFIT', 'SG_A', 'R_D', 'D_A', 'A_INTANGIBLES','NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT', 'TAX_EXPENSE','E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS', 'OTHER_ADJUSTMENTS','M_A_COST', 'NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI','RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES', 'DEFERRED_TAX', 'EBIT','EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR'], axis = 1)

***LIQUIDITY***

In [19]:
df_ratio3 = df_ratio.copy(deep = True)

In [20]:
df_ratio3['CURRENT_RATIO'] = round((df_ratio3['TOTAL_CURRENT_ASSETS'] - df_ratio3['INVENTORY']) / df_ratio3['CURRENT_LIAB'],2)
df_ratio3['QUICK_RATIO'] = round((df_ratio3['CASH'] + df_ratio3['RECEIVABLES'] + df_ratio3['ST_INVESTMENTS']) / df_ratio3['CURRENT_LIAB'],2)
df_ratio3['CASH_RATIO'] = round((df_ratio3['CASH'] + df_ratio3['ST_INVESTMENTS']) / df_ratio3['CURRENT_LIAB'],2)

In [21]:
liquidity = df_ratio3.drop(['CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY','OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION','LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS','NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT','CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES','OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB','RETAINED_EAR', 'TOTAL_EQUITY', 'TOTAL_REVENUES', 'COGS','GROSS_PROFIT', 'SG_A', 'R_D', 'D_A', 'A_INTANGIBLES','NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT', 'TAX_EXPENSE','E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS', 'OTHER_ADJUSTMENTS','M_A_COST', 'NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI','RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES', 'DEFERRED_TAX', 'EBIT','EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR'], axis = 1)

***PROFITABILITY***

In [22]:
df_ratio4 = df_ratio.copy(deep = True)

In [23]:
df_ratio4['GROSS_MARGIN'] = round(df_ratio4['GROSS_PROFIT'] / df_ratio4['TOTAL_REVENUES'],2)
df_ratio4['EBITDA_MARGIN'] = round(df_ratio4['CONSOLIDATED_EBITDA'] / df_ratio4['TOTAL_REVENUES'],2)
df_ratio4['EBIT_MARGIN'] = round(df_ratio4['EBIT'] / df_ratio4['TOTAL_REVENUES'],2)
df_ratio4['NET_MARGIN'] = round(df_ratio4['NET_INCOME_EXCL_EI'] / df_ratio4['TOTAL_REVENUES'],2)
df_ratio4['ROA'] = round(df_ratio4['TOTAL_REVENUES'] / df_ratio4['TOTAL_ASSETS'],2)
df_ratio4['ROE'] = round(df_ratio4['TOTAL_REVENUES'] / df_ratio4['TOTAL_EQUITY'],2)

In [24]:
profitability = df_ratio4.drop(['CASH', 'ST_INVESTMENTS', 'RECEIVABLES', 'INVENTORY','OTHER_CURRENT_ASSETS', 'TOTAL_CURRENT_ASSETS', 'PP&E', 'DEPRECIATION','LT_INVESTMENTS', 'INTANGIBLES_GW', 'OTHER_NON_CURRENT_ASSETS','NON_CURRENT_ASSETS', 'TOTAL_ASSETS', 'PAYABLES', 'ST_DEBT','CPLTD_DEBT', 'CPLTD_LEASES', 'CURRENT_LIAB', 'LT_DEBT', 'LT_LEASES','OTHER_NON_CURRENT_LIAB', 'NON_CURRENT_LIAB', 'TOTAL_LIAB','RETAINED_EAR', 'TOTAL_EQUITY', 'TOTAL_REVENUES', 'COGS','GROSS_PROFIT', 'SG_A', 'R_D', 'D_A', 'A_INTANGIBLES','NET_INTEREST_EXP', 'UNUSUAL_ITEMS', 'EBT', 'TAX_EXPENSE','E_CONT_OPER', 'E_DISC_OPER', 'EXTR_ITEMS', 'OTHER_ADJUSTMENTS','M_A_COST', 'NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI','RENTAL_EXPENSE', 'INTEREST_LT', 'TAXES', 'DEFERRED_TAX', 'EBIT','EBITDA', 'CONSOLIDATED_EBITDA', 'EBITDAR'], axis = 1)

## DESARROLLO DEL PROGRAMA

El programa fue elaborado con 'def function()', para evitar poner código adicional en el formato menú y fuera más fácil su elaboración

***la eliminación de cualquier celda podría afectar el funcionamiento del programa***


In [25]:
def login():
    login = str(input("BeFinData, please login: "))
    print(f"Welcome {login}! :)")
    print("----- MENU -----")

def Menu():
    global menu
    print()
    print("[1] Financial Information")
    print("[2] Financial Analysis")
    print("[3] Market information")
    print("[0] Exit")
    menu = int(input('-------Select one option:-------'))
    
def financial_infomenu():
    global select_data
    print("[1] Balance Sheet")
    print("[2] Income Statement")
    print("[3] Capital Structure")
    print("[0] Return to menu")
    print()
    select_data = int(input("""-----What Financial Statement do you want to print?:-----"""))
    
def time_print():
    print("[1] Select year from 2015 to 2020")
    print("[2] All years")
    print("[0] Return to menu")


In [26]:
def company_validate():
    global company
    global sector_company
    global industry_company
    company = input('Ticker Company: ')
    while company != 0:
        global company_name
        if company in ticker_symbol:   
            company_name = str(company_info['Company_Name'][company_info['Symbol'] == company].iloc[0])
            stock = str(company_info['Stock'][company_info['Symbol'] == company].iloc[0])
            sector_company = str(company_info['GICS_Sector'][company_info['Symbol'] == company].iloc[0])
            industry_company = str(company_info['GICS_SubIndustry'][company_info['Symbol'] == company].iloc[0])
            print(f"""------Company Information------

            Company Name: {company_name}
            Company Ticker: {company}
            Stock List: {stock}
            Sector: {sector_company}
            Industry: {industry_company}

            """)
            break;
        else:
            print("Incorrect Ticker")
            print()
            company = input('Ticker Company: ')
    print('-----------------')

**Continua estructura:**

In [27]:
def financial_print():
    global select_data
    financial_infomenu()
    while select_data != 0:   
        if select_data == 1:
            time_print()
            print()
            option = int(input("""-----Select time:-----"""))
            while option != 0:
                if option == 1:
                    print()
                    year = int(input('Select a Year (for 2015 to 2020 format: YYYY): '))
                    if year in list_year:
                        balance1 = balance.copy(deep = True)
                        filter_balance  = balance1[(balance1['Symbol'] == company) & (balance1['Year']==year)]
                        print()
                        print(f"""**Balance sheet of {company_name} year {year} was succesfully export**""")
                        filter_balance.to_csv(f'Datasets/Pruebas/balancesheet_{company_name}_{year}.csv', index = False, header=True)
                        break;
                    else:
                        print()
                        print('------Sorry my database is limited from 2015 to 2020------')
                        if year in list_year == True:
                            break;
                elif option == 2: 
                    balance1 = balance.copy(deep = True)
                    filter_balance  = balance1[balance1['Symbol'] == company]
                    print()
                    print(f"""Balance sheet of {company_name} from 2015 to 2020 was succesfully export""")
                    filter_balance.to_csv(f'Datasets/Pruebas/balancesheet_{company_name}_2015to2020.csv', index = False, header=True)
                    break;
                else:
                    print()
                    print('Invalid option')
                    break;
            print()
            print('******Thanks for using Balance Sheet App :) *******')
            print()
            print('Loading.......')
            print()
            financial_infomenu()
    ######AQUI ES EL CODIGO DE INCOME STATEMENT
        elif select_data == 2:
            time_print()
            print()
            option = int(input("""-----Select time:-----"""))  
            global income1
            while option != 0:
                if option == 1:
                    print()
                    year = int(input('Select a Year (for 2015 to 2020 format: YYYY): '))
                    if year in list_year:
                        income1 = income.copy(deep = True)
                        filter_income  = income1[(income1['Symbol'] == company) & (income1['Year']==year)]
                        print()
                        print(f"""**Income statement of {company_name} year {year} was succesfully export**""")
                        filter_income.to_csv(f'Datasets/Pruebas/incomestatement_{company_name}_{year}.csv', index = False, header=True)
                        break;
                    else:
                        print()
                        print('------Sorry my database is limited from 2015 to 2020------')
                        if year in list_year == True:
                            break;
                elif option == 2: 
                    income1 = income.copy(deep = True)
                    filter_income  = income1[income1['Symbol'] == company]
                    print()
                    print(f"""Income statement of {company_name} from 2015 to 2020 was succesfully export""")
                    filter_income.to_csv(f'Datasets/Pruebas/incomestatement_{company_name}_2015to2020.csv', index = False, header=True)
                    break;
                else:
                    print()
                    print('Invalid option')
                    break;
            print()
            print('******Thanks for using Income Statement App :) *******')
            print()
            print('Loading.......')
            print()
            financial_infomenu()

        elif select_data == 3:
            time_print()
            print()
            option = int(input("""-----Select time:-----"""))
            global capital_structure1
            while option != 0:
                if option == 1:
                    print()
                    year = int(input('Select a Year (for 2015 to 2020 format: YYYY): '))
                    if year in list_year:
                        capital_structure1 = capital_structure.copy(deep = True)
                        filter_capital_structure  = capital_structure1[(capital_structure1['Symbol'] == company) & (capital_structure1['Year']==year)]
                        print()
                        print(f"""**Capital structure of {company_name} year {year} was succesfully export**""")
                        filter_capital_structure.to_csv(f'Datasets/Pruebas/capitalstructure_{company_name}_{year}.csv', index = False, header=True)
                        if True:
                            break;
                        if False:
                            continue;
                    else:
                        print()
                        print('------Sorry my database is limited from 2015 to 2020------')
                        if year in list_year == True:
                            break;
                elif option == 2:  
                    capital_structure1 = capital_structure.copy(deep = True)
                    filter_capital_structure  = capital_structure1[capital_structure1['Symbol'] == company]
                    print()
                    print(f"""Capital structure of {company_name} from 2015 to 2020 was succesfully export""")
                    filter_capital_structure.to_csv(f'Datasets/Pruebas/capitalstructure_{company_name}_2015to2020.csv', index = False, header=True)
                    break;
                else:
                    print()
                    print('Invalid option')
                    break;
            print()
            print('******Thanks for using Capital Structure App :) *******')
            print()
            print('Loading.......')
            print()
            financial_infomenu()
        else:
            print('------Invalid option-------')
            print()
            financial_infomenu()

    print('Thanks for using this app')

**Continua estructura:**

In [28]:
def balance_compa():
    balance_company = balance.copy(deep = True) 
    balance_filter = balance_company[balance_company['Symbol'] == company]
    balance_filter = balance_filter[['Year','Symbol', "TOTAL_ASSETS", 'TOTAL_LIAB', 'TOTAL_EQUITY']]
    balance_filter = balance_filter.rename(columns = {"TOTAL_ASSETS":'Assets', 'TOTAL_LIAB':'Liabilities', 'TOTAL_EQUITY':'Equity'})
    fig = go.Figure(data=[
        go.Bar(name='Assets', x=balance_filter['Year'], y=balance_filter['Assets'], marker_color = '#1434A4', text = balance_filter['Assets']),
        go.Bar(name='Liabilities', x=balance_filter['Year'], y=balance_filter['Liabilities'], marker_color = '#89CFF0', text = balance_filter['Liabilities']),
        go.Bar(name='Equity', x=balance_filter['Year'], y=balance_filter['Equity'], marker_color = '#40B5AD', text = balance_filter['Equity'])
    ])
    fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
    fig.update_layout(title=f'Balance sheet of {company_name}')
    fig.update_xaxes(title_text='Years')
    fig.update_yaxes(title_text='"000" USD')
    fig.show()

In [37]:
def income_compa():
    income_company = income.copy(deep = True) 
    income_filter = income_company[income_company['Symbol'] == company]
    income_filter = income_filter[['Year','Symbol', "TOTAL_REVENUES", 'GROSS_PROFIT', 'CONSOLIDATED_EBITDA','EBIT','EBT','NET_INCOME_INCL_EI', 'NET_INCOME_EXCL_EI']]
    income_filter = income_filter.rename(columns = {"TOTAL_REVENUES":'Revenues', 'GROSS_PROFIT':'Gross Profit','CONSOLIDATED_EBITDA':'EBITDA','NET_INCOME_INCL_EI':'Net Income + Ext Items','NET_INCOME_EXCL_EI':'Net Income'})
    income_filter = income_filter.sort_values(by = 'Year')
    fig = go.Figure()
    fig.add_trace(go.Scatter(name='Revenues', x=income_filter['Year'], y=income_filter['Revenues'], marker_color = '#f04652'))
    fig.add_trace(go.Scatter(name='Gross Profit', x=income_filter['Year'], y=income_filter['Gross Profit'], marker_color = '#082b5e', text = income_filter['Gross Profit']))
    fig.add_trace(go.Scatter(name='EBITDA', x=income_filter['Year'], y=income_filter['EBITDA'], marker_color = '#1167b1'))
    fig.add_trace(go.Scatter(name='EBIT', x=income_filter['Year'], y=income_filter['EBIT'], marker_color = '#93a0ee'))
    fig.add_trace(go.Scatter(name='EBT', x=income_filter['Year'], y=income_filter['EBT'], marker_color = '#a26fa0'))
    fig.add_trace(go.Scatter(name='Net Income + Ext Items', x=income_filter['Year'], y=income_filter['Net Income + Ext Items'], marker_color = '#4cefb9'))
    fig.add_trace(go.Scatter(name='Net Income', x=income_filter['Year'], y=income_filter['Net Income'], marker_color = '#25b579'))
    fig.update_layout(title=f'Income statement of {company_name} / Principal items')
    fig.update_xaxes(title_text='Years')
    fig.update_yaxes(title_text='"000" USD')
    fig.show()

In [39]:
def evolution_debt ():
    lev = leverage.copy(deep = True)
    lev_sub = lev.merge(comp[['Symbol','GICS_SubIndustry']], on = "Symbol", how = 'left')
    lev_sub_nf = lev_sub[(lev_sub['Sector_x'] != 'Financials') & (lev_sub['TOTAL_DEBT'] != 0)]
    evo_debt = lev_sub_nf.groupby(["Sector_x",'Year']).sum()
    sns.set_theme(style="darkgrid", font_scale =1)
    plt.figure(figsize = (10, 8))
    evodebt = sns.lineplot(x = 'Year', y = 'TOTAL_DEBT', data = evo_debt, hue = 'Sector_x', linewidth = 3)
    evodebt.set_title('Evolution of debt in S&P 500', size = 23, fontweight = 'bold')
    evodebt.set_xlabel('Year', size = 15, fontweight = 'bold')
    evodebt.set_ylabel('Debt in USD (000)', size = 15, fontweight = 'bold')
    evodebt.legend(title = 'Sector', loc = 2, bbox_to_anchor = (1,1), fontsize = 12, title_fontsize = 15)
    ylabels = ['{:,.0f}'.format(x) + 'K' for x in evodebt.get_yticks()/1000]
    evodebt.set_yticklabels(ylabels, size = 12)
    xlabels = ['{:.0f}'.format(x) for x in evodebt.get_xticks()]
    evodebt.set_xticklabels(xlabels, size = 12)
    evodebt

In [31]:
def statistics_industry():
    df_lev = df_ratio.copy(deep = True)
    df_lev = df_lev[(df_lev['Sector_x'] == sector_company) & (df_lev['Year'] == year) & (df_lev['GICS_SubIndustry'] == industry_company)]
    df_lev = df_lev.rename(columns = {'TOTAL_DEBT':'Debt',"TOTAL_REVENUES":'Revenues', 'GROSS_PROFIT':'Gross Profit','CONSOLIDATED_EBITDA':'EBITDA_CON','NET_INCOME_INCL_EI':'Net Income + Ext Items','NET_INCOME_EXCL_EI':'Net Income'})
    fig = go.Figure()
    fig.add_trace(go.Box(y=df_lev['Revenues'], quartilemethod="linear", name= 'Revenues'))
    fig.add_trace(go.Box(y=df_lev['EBITDA_CON'], quartilemethod='linear', name= 'EBITDA'))
    fig.add_trace(go.Box(y=df_lev['Net Income'], quartilemethod='linear', name= 'Net Income'))
    fig.add_trace(go.Box(y=df_lev['CASH'], quartilemethod='linear', name= 'Cash'))
    fig.add_trace(go.Box(y=df_lev['INVENTORY'], quartilemethod='linear', name= 'Inventory'))
    fig.add_trace(go.Box(y=df_lev['RECEIVABLES'], quartilemethod='linear', name= 'Receivables'))
    fig.add_trace(go.Box(y=df_lev['INTANGIBLES_GW'], quartilemethod='linear', name= 'Goodwill'))
    fig.add_trace(go.Box(y=df_lev['PAYABLES'], quartilemethod='linear', name= 'Payables'))
    fig.add_trace(go.Box(y=df_lev['Debt'], quartilemethod='linear', name= 'Debt'))
    fig.add_trace(go.Box(y=df_lev['TOTAL_EQUITY'], quartilemethod='linear', name= 'Equity'))
    fig.update_layout(
        title=go.layout.Title(
            text=f"Statistics for sector: {sector_company} for year {year}  <br><sup>{industry_company}</sup>",
            xref="paper",
            x=0
        ))
    fig.update_yaxes(title_text='"000" USD')
    fig.show()

In [32]:
def complete_ratios():
    ratios_all = reduce(lambda x,y: pd.merge(x,y, on=['Symbol', 'Year'], how='outer'), [leverage, coverage, liquidity, profitability])
    ratios_all.dtypes
    ratios_all = ratios_all[['Symbol','Year','Company_Name_x_x','Sector_x_y','GICS_SubIndustry_x','TOTAL_DEBT_x','DEBT_TO_ASSETS','DEBT_TO_EQUITY','DEBT_TO_CAPITAL','DEBT_TO_EBITDA','DEBT_TO_EBITDAR','ASSET_TO_EQUITY','LIABILITIES_TO_EQUITY','INTEREST_COVERAGE','DEBT_SERVICE_COVERAGE','CASH_COVERAGE','ASSET_COVERAGE','CURRENT_RATIO','QUICK_RATIO','CASH_RATIO','EBITDA_MARGIN','GROSS_MARGIN','EBIT_MARGIN','NET_MARGIN','ROA','ROE']]
    ratios_all = ratios_all.loc[:,~ratios_all.columns.duplicated()]
    ratios_all = ratios_all.rename(columns = {'Company_Name_x_x':'Company Name', 'Sector_x_y':'Sector', 'GICS_SubIndustry_x':'Industry'})
    count_nan = ratios_all.isnull().sum()
    count_nan
    ratios_all_comp = ratios_all[(ratios_all['Sector'] == sector_company) & (ratios_all['Year'] == year) & (ratios_all['Industry'] == industry_company)]
    #fig = px.bar(ratios_all_comp, y='DEBT_TO_ASSETS', x='Company Name', text='DEBT_TO_ASSETS')
    fig = go.Figure(data=[
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['LIABILITIES_TO_EQUITY'], text = ratios_all_comp['LIABILITIES_TO_EQUITY'], name = 'Liabilities to equity'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['ASSET_TO_EQUITY'], text = ratios_all_comp['ASSET_TO_EQUITY'], name = 'Assets to Equity'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['DEBT_TO_EBITDA'], text = ratios_all_comp['DEBT_TO_EBITDA'], name = 'Debt to EBITDA')
    ])
    fig.update_traces(texttemplate='%{text:.2}x', textposition='outside')
    fig.update_layout(
            title=go.layout.Title(
                text=f"Leverage Ratios and comparables for: {company_name} year {year}  <br><sup>Sector: {sector_company}, Industry: {industry_company}</sup>",
                xref="paper",
                x=0
            ), uniformtext_minsize=5, uniformtext_mode='hide')
    fig.update_yaxes(title_text='Times')
    fig.show()
    #####COVERAGE
    fig2 = go.Figure(data=[
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['INTEREST_COVERAGE'], text = ratios_all_comp['INTEREST_COVERAGE'], name = 'Interest Coverage'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['DEBT_SERVICE_COVERAGE'], text = ratios_all_comp['DEBT_SERVICE_COVERAGE'], name = 'Debt Service Coverage'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['CASH_COVERAGE'], text = ratios_all_comp['CASH_COVERAGE'], name = 'Cash Coverage'),
    ])
    fig2.update_traces(texttemplate='%{text:.2}x', textposition='outside')
    fig2.update_layout(
            title=go.layout.Title(
                text=f"Coverage Ratios and comparables for: {company_name} year {year}  <br><sup>Sector: {sector_company}, Industry: {industry_company}</sup>",
                xref="paper",
                x=0
            ), uniformtext_minsize=5, uniformtext_mode='hide')
    fig2.update_yaxes(title_text='Times')
    fig2.show()

    ####liquidity
    fig3 = go.Figure(data=[
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['CURRENT_RATIO'], text = ratios_all_comp['CURRENT_RATIO'], name = 'Current Ratio'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['QUICK_RATIO'], text = ratios_all_comp['QUICK_RATIO'], name = 'Quick Ratio'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['CASH_RATIO'], text = ratios_all_comp['CASH_RATIO'], name = 'Cash Ratio'),
    ])
    fig3.update_traces(texttemplate='%{text:.2}x', textposition='outside')
    fig3.update_layout(
            title=go.layout.Title(
                text=f"Liquidity Ratios and comparables for: {company_name} year {year}  <br><sup>Sector: {sector_company}, Industry: {industry_company}</sup>",
                xref="paper",
                x=0
            ), uniformtext_minsize=5, uniformtext_mode='hide')
    fig3.update_yaxes(title_text='Times')
    fig3.show()

    ####Profitability
    fig4 = go.Figure(data=[
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['GROSS_MARGIN'], text = ratios_all_comp['GROSS_MARGIN'], name = 'Gross Margin'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['EBITDA_MARGIN'], text = ratios_all_comp['EBITDA_MARGIN'], name = 'EBITDA Margin'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['EBIT_MARGIN'], text = ratios_all_comp['EBIT_MARGIN'], name = 'EBIT Margin'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['NET_MARGIN'], text = ratios_all_comp['NET_MARGIN'], name = 'Net Margin'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['ROA'], text = ratios_all_comp['ROA'], name = 'ROA'),
        go.Bar(x=ratios_all_comp['Company Name'], y=ratios_all_comp['ROE'], text = ratios_all_comp['ROE'], name = 'ROE')
    ])
    fig4.update_traces(texttemplate='%{text:.2}%', textposition='outside')
    fig4.update_layout(
            title=go.layout.Title(
                text=f"Profitability Ratios and comparables for: {company_name} year {year}  <br><sup>Sector: {sector_company}, Industry: {industry_company}</sup>",
                xref="paper",
                x=0
            ), uniformtext_minsize=5, uniformtext_mode='hide')
    fig4.update_yaxes(title_text='Percentage (1 = 100%)')
    fig4.show()

In [33]:
def validate_year():    
    global year
    year = int(input('Select a Year (for 2015 to 2020 format: YYYY): '))
    while year != 0:
        if year in list_year:
            break;
        else:
            print()
            print('Sorry my data base is limited from 2015 to 2020')
            print()
            print('Try with this years: 2015, 2016, 2017, 2018, 2019, 2020')
            print()
        year = int(input('Select a Year (for 2015 to 2020 format: YYYY): '))


# ESTE ES EL PROGRAMA / ANÁLISIS:

In [40]:
login()
Menu()
while menu != 0:
    if menu == 1:
        company_validate()
        while True:
            financial_print()
            end = int(input('Exit?: [1] No [2] Yes'))
            if end != 2:
                break;
            break;
        if end == 2:
            Menu()
    elif menu == 2:
        company_validate()
        print()
        validate_year()
        print()
        while True:
            balance_compa()
            income_compa()
            statistics_industry()
            complete_ratios()
            end = int(input('Exit?: [1] No [2] Yes'))
            if end != 2:
                break;
            break;
        if end == 2:
            Menu()
    elif menu == 3:
        evolution_debt()
        Menu()
    else:
        print('Incorrect number')
        Menu()
print('See you soon :)')

BeFinData, please login:  1


Welcome 1! :)
----- MENU -----

[1] Financial Information
[2] Financial Analysis
[3] Market information
[0] Exit


-------Select one option:------- 3


NameError: name 'comp' is not defined

# Futuros pasos:

Se considera en algún futuro cercano lo siguiente:
1. Ocupar herramientas de Machine Learning para desarrollar un análisis de crédito y asignar alguna calificación crediticia
2. Ocupar los dataset disponibles para generar una valuación (falta el dataset de cash flow)
3. Conectar a alguna API de información financiera y poder generar algunos modelos para precio de la acción futuro, considerando cruzar con algunas otras variables
4. Sigo considerando nuevas ideas jajaja



# Gracias por su atención :)